# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откроем и изучим файл

In [1]:
# импортируем библиотеки 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.shape

(3214, 5)

In [5]:
# пропущенные значения
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [6]:
# дубликаты
df.duplicated().sum()

0

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [8]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


####  Датафрейм содержит данные  о поведении  3214 клиентов, поведение каждого клиента  описывается 5 параметрами. Имена столбцов заданы корректно, типы данных в столбцах в порядке. Дубликатов строк и пропущенных значений в столбцах нет.

## Разобьем данные на выборки

In [9]:
# 60% - обучающая выборка, 40% - валидационная + тестовая 
df_train, df_valid_test = train_test_split(df, test_size=0.4, random_state=12345)
# матрица исходных данных и целевой столбец для обучайющей выборки
features_train = df_train.drop('is_ultra', axis=1) 
target_train = df_train['is_ultra'] 
# матрица исходных данных и целевой столбец для валидационной + тестовой выборки
features_valid_test = df_valid_test.drop('is_ultra', axis=1) 
target_valid_test = df_valid_test['is_ultra']
# разделим на тестовую и валидационную
features_valid, features_test, target_valid, target_test = train_test_split(features_valid_test, target_valid_test,
                                                                            test_size=0.5, random_state=12345)

#### Разбили данные на обучающую (60%),валидационную (20%) и тестовою (20%) выборки. Разделили  каждую  выборку на  матрицу исходных данных и целевую переменную.
* features_train, target_train - обучающая выборка 
* features_valid, target_valid - валидационная выборка 
* features_test, target_test - тестовая выборка 

## Исследуем модели

###  Метод Decision Tree

In [10]:
best_model_DT = None
best_result_DT = 0
for depth in range(1, 26):
    model = DecisionTreeClassifier(random_state=12345, max_depth= depth) # обучение модели с заданной глубиной дерева
    model.fit(features_train, target_train) # обучение модели
    result = model.score(features_valid, target_valid)
    if result > best_result_DT:
        best_model_DT = model
        best_result_DT = result
        best_depth_DT = depth
        
print("Глубина дерева:", best_depth_DT, " Accuracy лучшей модели:", best_result_DT)
    

Глубина дерева: 3  Accuracy лучшей модели: 0.7853810264385692


### Метод Random Forest

In [11]:
best_model_RF = None
best_result_RF = 0
for est in range(5,50,5):
    for depth in range(1, 10):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)
        if result > best_result_RF:
            best_model_RF = model
            best_result_RF = result
            best_est_RF = est
            best_depth_RF = depth
        
print("Количество деревьев:", best_est_RF, " Глубина дерева:", best_depth_RF, " Accuracy лучшей модели:", best_result_RF)               

Количество деревьев: 40  Глубина дерева: 8  Accuracy лучшей модели: 0.8087091757387247


### Метод Logistic Regression

In [12]:
model_LR = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_LR.fit(features_train, target_train)
accuracy_LR = model_LR.score(features_valid, target_valid)
print("Accuracy:", accuracy_LR)

Accuracy: 0.7107309486780715


#### На валидациооной выборке  наилучший прогноз дает Random Forest,  согласно показателя Accuracy. 

## Проверим модель на тестовой выборке

In [13]:
print("Accuracy Random Forest:", best_model_RF.score(features_test, target_test))

Accuracy Random Forest: 0.7962674961119751


#### На тестовой выборке наша модель Random Forest показывает точность 0.796,  достигается при использовании  гиперпараметрамов max_depth=8 , n_estimators=40

## Проверим модели на адекватность

### Посчитаем фактическую долю клиентов использующих тариф Ультра и сравним с предсказаниями наших моделей. Исследования проведем на тестовой выборке.

In [14]:
is_ultra_fact = pd.Series(target_test).mean() # фактическая доля клиентов
print("Фактическая доля клиентов:",is_ultra_fact)
is_ultra_LR = model_LR.predict(features_test).mean()
print("Предсказания Logistic Regressionis:",is_ultra_LR)
is_ultra_DT = best_model_DT.predict(features_test).mean()
print("Предсказания Decision Tree:", is_ultra_DT)
is_ultra_RF = best_model_RF.predict(features_test).mean()
print("Предсказания Random Forest:",is_ultra_RF)

Фактическая доля клиентов: 0.3157076205287714
Предсказания Logistic Regressionis: 0.024883359253499222
Предсказания Decision Tree: 0.19440124416796267
Предсказания Random Forest: 0.223950233281493


#### Наилучшие предсказания по доли клиентов использующих тариф Ультра дает модель Random Forest. Предсказания модели Logistic Regressionis значительно отличаются от фактического значения. Модель Logistic Regressionis нельзя назвать адекватной.  
